In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd '/content/drive/MyDrive/csci544-project/'

In [ ]:
# import necessary packages
# %tensorflow_version 2.x
import json
import time
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
from numpy import newaxis
# import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow import keras
from keras.layers import Dense, Activation, Dropout, LSTM, GRU
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

from math import pi,sqrt,exp,pow,log
from numpy.linalg import det, inv
from abc import ABCMeta, abstractmethod
from sklearn import cluster

import statsmodels.api as sm 
import scipy.stats as scs
import scipy.optimize as sco
import scipy.interpolate as sci
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_curve, auc, mean_squared_error

# Level 0 - Second Model

In [3]:
# read data - local and colab
# df1 = pd.read_csv("/Users/nathanli/masterProject/env/data/DP-LSTM_DATA/source_price.csv")
url = './data/source_price_sentiment_oecd.csv'
df2= pd.read_csv(url)
df2.tail()

,date,wsj_mean_compound,cnbc_mean_compound,fortune_mean_compound,reuters_mean_compound,Adj Close,CLI,BCI,CCI
116,2018/5/25,0.030290,0.047433,0.011550,-0.025190,2721.330078,0.7426,1.2187,1.5006
117,2018/5/29,-0.052796,0.070442,-0.025721,-0.035568,2689.860107,0.7426,1.2187,1.5006
118,2018/5/30,-0.017367,0.038119,-0.076965,-0.063177,2724.010010,0.7426,1.2187,1.5006
119,2018/5/31,-0.018636,0.057371,-0.064138,-0.025489,2705.270020,0.7426,1.2187,1.5006
120,2018/6/1,0.000000,-0.061150,0.361200,-0.004489,2734.620117,0.7163,1.3308,1.4417


In [4]:
# set index to date
df2.index = df2.date
df2.drop('date', axis=1, inplace=True)
df2

,wsj_mean_compound,cnbc_mean_compound,fortune_mean_compound,reuters_mean_compound,Adj Close,CLI,BCI,CCI
date,,,,,,,,
2017/12/7,0.296000,-0.136600,0.000000,0.000000,2636.979980,0.4484,1.3452,1.3407
2017/12/8,0.000000,0.000000,-0.242300,0.000000,2651.500000,0.4484,1.3452,1.3407
2017/12/11,0.000000,0.000000,0.000000,0.000000,2659.989990,0.4484,1.3452,1.3407
2017/12/12,0.000000,0.000000,0.000000,0.000000,2664.110107,0.4484,1.3452,1.3407
2017/12/13,0.000000,0.000000,0.000000,0.000000,2662.850098,0.4484,1.3452,1.3407
...,...,...,...,...,...,...,...,...
2018/5/25,0.030290,0.047433,0.011550,-0.025190,2721.330078,0.7426,1.2187,1.5006
2018/5/29,-0.052796,0.070442,-0.025721,-0.035568,2689.860107,0.7426,1.2187,1.5006
2018/5/30,-0.017367,0.038119,-0.076965,-0.063177,2724.010010,0.7426,1.2187,1.5006


In [5]:
adjust_close=df2['Adj Close'].values
CLI=df2['CLI'].values
BCI=df2['BCI'].values
CCI=df2['CCI'].values
adjust_close

array([2636.97998 , 2651.5     , 2659.98999 , 2664.110107, 2662.850098,
       2652.01001 , 2675.810059, 2690.159912, 2681.469971, 2679.25    ,
       2684.570068, 2683.340088, 2680.5     , 2682.620117, 2687.540039,
       2673.610107, 2695.810059, 2713.060059, 2723.98999 , 2743.149902,
       2747.709961, 2751.290039, 2748.22998 , 2767.560059, 2786.23999 ,
       2776.419922, 2802.560059, 2798.030029, 2810.300049, 2832.969971,
       2839.129883, 2837.540039, 2839.25    , 2872.870117, 2853.530029,
       2822.429932, 2823.810059, 2821.97998 , 2762.129883, 2648.939941,
       2695.139893, 2681.659912, 2581.      , 2619.550049, 2656.      ,
       2662.939941, 2698.629883, 2731.199951, 2732.219971, 2716.26001 ,
       2701.330078, 2703.959961, 2747.300049, 2779.600098, 2744.280029,
       2713.830078, 2677.669922, 2691.25    , 2720.939941, 2728.120117,
       2726.800049, 2738.969971, 2786.570068, 2783.02002 , 2765.310059,
       2749.47998 , 2747.330078, 2752.01001 , 2712.919922, 2716.

In [6]:
# nomalize adjust close stock price
scaler = MinMaxScaler()
scaled_adjust_close=scaler.fit_transform(adjust_close.reshape(-1,1))

In [7]:
# check if adjust close is scaled
scaled_adjust_close

array([[0.19179757],
       [0.2415458 ],
       [0.27063404],
       [0.28475031],
       [0.28043329],
       [0.24329318],
       [0.32483647],
       [0.37400167],
       [0.34422836],
       [0.33662233],
       [0.35484985],
       [0.35063572],
       [0.34090506],
       [0.34816897],
       [0.36502551],
       [0.31729904],
       [0.3933601 ],
       [0.45246173],
       [0.48990966],
       [0.555555  ],
       [0.57117859],
       [0.58344458],
       [0.57296027],
       [0.63918863],
       [0.70318946],
       [0.66954412],
       [0.75910498],
       [0.74358427],
       [0.78562359],
       [0.86329486],
       [0.88439983],
       [0.87895274],
       [0.88481138],
       [1.        ],
       [0.93373735],
       [0.82718277],
       [0.83191134],
       [0.82564115],
       [0.62058386],
       [0.23277457],
       [0.39106399],
       [0.34487913],
       [0.        ],
       [0.13207947],
       [0.25696361],
       [0.28074111],
       [0.40302133],
       [0.514

In [8]:
new_data=df2.drop(['Adj Close'],axis=1)

new_data['scaled_adj_close'] = scaled_adjust_close

new_data.tail()

,wsj_mean_compound,cnbc_mean_compound,fortune_mean_compound,reuters_mean_compound,CLI,BCI,CCI,scaled_adj_close
date,,,,,,,,
2018/5/25,0.030290,0.047433,0.011550,-0.025190,0.7426,1.2187,1.5006,0.480796
2018/5/29,-0.052796,0.070442,-0.025721,-0.035568,0.7426,1.2187,1.5006,0.372974
2018/5/30,-0.017367,0.038119,-0.076965,-0.063177,0.7426,1.2187,1.5006,0.489978
2018/5/31,-0.018636,0.057371,-0.064138,-0.025489,0.7426,1.2187,1.5006,0.425772
2018/6/1,0.000000,-0.061150,0.361200,-0.004489,0.7163,1.3308,1.4417,0.526330


In [9]:
new_data.dtypes

wsj_mean_compound        float64
cnbc_mean_compound       float64
fortune_mean_compound    float64
reuters_mean_compound    float64
CLI                      float64
BCI                      float64
CCI                      float64
scaled_adj_close         float64
dtype: object

In [10]:
new_data.shape

(121, 8)

In [12]:
# split data into train and test
# train 85% and test 15%
train_validation_data, test_data= train_test_split(new_data, train_size=0.85, test_size=0.15,shuffle=False)

In [13]:
train_validation_data.tail()

,wsj_mean_compound,cnbc_mean_compound,fortune_mean_compound,reuters_mean_compound,CLI,BCI,CCI,scaled_adj_close
date,,,,,,,,
2018/4/30,-0.009474,0.064056,0.021328,0.066076,0.7434,1.1734,1.5880,0.229726
2018/5/1,-0.044864,0.084246,-0.013291,0.068956,0.7426,1.2187,1.5006,0.252852
2018/5/2,0.000608,0.088057,0.043339,0.053706,0.7426,1.2187,1.5006,0.187309
2018/5/3,0.017188,0.066461,0.063148,0.068072,0.7426,1.2187,1.5006,0.166958
2018/5/4,-0.001986,0.073777,-0.021277,0.062085,0.7426,1.2187,1.5006,0.282386


In [14]:
train_validation_data.shape

(102, 8)

In [15]:
validation_data =train_validation_data[83:]
train_data=train_validation_data[:83]

print("train data shape: ", train_data.shape)
print("validation data shape: ", validation_data.shape)
print("test data shape: ", test_data.shape)

train data shape:  (83, 8)
validation data shape:  (19, 8)
test data shape:  (19, 8)


In [16]:
# configs
look_back = 10
forward_days = 1
NUM_NEURONS=50
EPOCHES=100
BATCH_SIZE=32
DROUP_OUT=0.2

In [17]:
#Get the data and splits in input X and output Y, by spliting in `n` past days as input X 
#and `m` coming days as Y.
def processData(data, look_back, forward_days,jump=1):
    A,B = [],[]
    for i in range(0,len(data) -look_back, jump):
        A.append(data[i:(i+look_back)])
        B.append(data[(i+look_back):(i+look_back+forward_days)])
    return np.array(A), np.array(B)

In [18]:
m_training_data = train_data.to_numpy()
m_validation_data = validation_data.to_numpy()
m_test_data = test_data.to_numpy()

X_train,Y_train = processData(m_training_data,look_back,forward_days)
X_validation,Y_validation = processData(m_validation_data,look_back,forward_days)
X_test,Y_test = processData(m_test_data,look_back,forward_days)

In [19]:
Y_train=Y_train[:,0,-1]
Y_validation=Y_validation[:,0,-1]
Y_test=Y_test[:,0,-1]

print("This is y_train shape",Y_train.shape)
print("This is y_validation shape",Y_validation.shape)
print("This is y_test shape",Y_test.shape)

This is y_train shape (73,)
This is y_validation shape (9,)
This is y_test shape (9,)


# LSTM model

In [24]:
# build LSTM model
model1 = Sequential()
# first layer
model1.add(LSTM(units=NUM_NEURONS, return_sequences = True, input_shape=(X_train.shape[1],X_train.shape[2])))
model1.add(Dropout(DROUP_OUT))
# second layer
model1.add(LSTM(units=NUM_NEURONS, return_sequences = True))
model1.add(Dropout(DROUP_OUT))
# third layer
model1.add(LSTM(units=NUM_NEURONS, return_sequences = True))
model1.add(Dropout(DROUP_OUT))
# fourth layer
model1.add(LSTM(units=NUM_NEURONS))
model1.add(Dropout(DROUP_OUT))
# outut layer
model1.add(Dense(forward_days))
#train model
model1.compile(loss='mean_squared_error',optimizer='adam')
              
              # compile
              
history1 =model1.fit(X_train, Y_train, epochs = EPOCHES, batch_size = BATCH_SIZE)

Train on 73 samples
Epoch 1/100
73/73 [==============================] - 1s 18ms/sample - loss: 0.2610
Epoch 2/100
73/73 [==============================] - 0s 942us/sample - loss: 0.1405
Epoch 3/100
73/73 [==============================] - 0s 940us/sample - loss: 0.0727
Epoch 4/100
73/73 [==============================] - 0s 996us/sample - loss: 0.0847
Epoch 5/100
73/73 [==============================] - 0s 1ms/sample - loss: 0.0578
Epoch 6/100
73/73 [==============================] - 0s 940us/sample - loss: 0.0706
Epoch 7/100
73/73 [==============================] - 0s 891us/sample - loss: 0.0663
Epoch 8/100
73/73 [==============================] - 0s 985us/sample - loss: 0.0645
Epoch 9/100
73/73 [==============================] - 0s 917us/sample - loss: 0.0610
Epoch 10/100
73/73 [==============================] - 0s 964us/sample - loss: 0.0647
Epoch 11/100
73/73 [==============================] - 0s 967us/sample - loss: 0.0563
Epoch 12/100
73/73 [==============================] - 0s 

In [25]:
# build GRU model
model2 = Sequential()
# first layer
model2.add(GRU(units=NUM_NEURONS, return_sequences = True, input_shape=(X_train.shape[1],X_train.shape[2])))
model2.add(Dropout(DROUP_OUT))
# second layer
model2.add(GRU(units=NUM_NEURONS, return_sequences = True))
model2.add(Dropout(DROUP_OUT))
# third layer
model2.add(GRU(units=NUM_NEURONS, return_sequences = True))
model2.add(Dropout(DROUP_OUT))
# fourth layer
model2.add(GRU(units=NUM_NEURONS))
model2.add(Dropout(DROUP_OUT))
# output layer
model2.add(Dense(forward_days))
#train model
model2.compile(loss='mean_squared_error',optimizer='rmsprop')
              
              # compile
              
history2 =model2.fit(X_train, Y_train, epochs = EPOCHES, batch_size = BATCH_SIZE)

Train on 73 samples
Epoch 1/100
73/73 [==============================] - 1s 20ms/sample - loss: 0.1981
Epoch 2/100
73/73 [==============================] - 0s 856us/sample - loss: 0.0608
Epoch 3/100
73/73 [==============================] - 0s 1ms/sample - loss: 0.0978
Epoch 4/100
73/73 [==============================] - 0s 879us/sample - loss: 0.0636
Epoch 5/100
73/73 [==============================] - 0s 943us/sample - loss: 0.0604
Epoch 6/100
73/73 [==============================] - 0s 902us/sample - loss: 0.0579
Epoch 7/100
73/73 [==============================] - 0s 867us/sample - loss: 0.0518
Epoch 8/100
73/73 [==============================] - 0s 937us/sample - loss: 0.0916
Epoch 9/100
73/73 [==============================] - 0s 897us/sample - loss: 0.0497
Epoch 10/100
73/73 [==============================] - 0s 902us/sample - loss: 0.0768
Epoch 11/100
73/73 [==============================] - 0s 1ms/sample - loss: 0.0497
Epoch 12/100
73/73 [==============================] - 0s 92

# Attention

In [21]:
# attention (+LSTM)
%pip install keras-self-attention
from keras_self_attention import SeqSelfAttention

model3 = Sequential()
model3.add(LSTM(units=NUM_NEURONS, return_sequences = True, input_shape=(X_train.shape[1],X_train.shape[2])))
model3.add(Dropout(DROUP_OUT))
model3.add(SeqSelfAttention(attention_activation='sigmoid',attention_width=3, history_only=True, attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL))
model3.add(LSTM(units=NUM_NEURONS))
model3.add(Dropout(DROUP_OUT))
model3.add(Dense(forward_days))
model3.compile(loss='mean_squared_error', optimizer='adam')
history3 =model3.fit(X_train, Y_train, epochs = EPOCHES, batch_size = BATCH_SIZE)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Train on 73 samples
Epoch 1/100
73/73 [==============================] - 1s 14ms/sample - loss: 0.1822
Epoch 2/100
73/73 [==============================] - 0s 549us/sample - loss: 0.0640
Epoch 3/100
73/73 [==============================] - 0s 549us/sample - loss: 0.0961
Epoch 4/100
73/73 [==============================] - 0s 698us/sample - loss: 0.0787
Epoch 5/100
73/73 [==============================] - 0s 597us/sample - loss: 0.0598
Epoch 6/100
73/73 [==============================] - 0s 600us/sample - loss: 0.0594
Epoch 7/100
73/73 [==============================] - 0s 661us/sample - loss: 0.0594
Epoch 8/100
73/73 [==============================] - 0s 602us/sample - loss: 0.0503
Epoch 9/100
73/73 [==============================] - 0s 707us/sample - loss: 0.0464
Epoch 10/100
73/73 [==============================] - 0s 805us/sample - loss: 0.0609
Epoch 11/100
73/73 [=====================

In [28]:
Xm1=model1.predict(X_validation)
Xm1.shape

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


(9, 1)

In [29]:
Xm2=model2.predict(X_validation)
Xm2.shape

(9, 1)

In [22]:
Xm3=model3.predict(X_validation)
Xm3.shape

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


(9, 1)

In [30]:
print(Xm1)
print(Xm2)
print(Xm3)

[[0.30414674]
 [0.3239251 ]
 [0.3270585 ]
 [0.31582546]
 [0.27470827]
 [0.24513344]
 [0.23315614]
 [0.2519112 ]
 [0.28723434]]
[[0.3205406 ]
 [0.21276852]
 [0.14849275]
 [0.15027365]
 [0.19846721]
 [0.23930892]
 [0.27147412]
 [0.2735144 ]
 [0.2511515 ]]
[[0.3889637 ]
 [0.38021547]
 [0.3454618 ]
 [0.30505025]
 [0.2695878 ]
 [0.25908905]
 [0.26891118]
 [0.2950235 ]
 [0.3222078 ]]


In [31]:
Y_validation = Y_validation.reshape(-1,1)
m2_Y_validation = scaler.inverse_transform(Y_validation)

In [32]:
Y_validation.reshape(-1,1)
Y_validation.shape

(9, 1)

In [33]:
m2_Y_validation

array([[2634.560059],
       [2639.399902],
       [2666.939941],
       [2669.909912],
       [2648.050049],
       [2654.800049],
       [2635.669922],
       [2629.72998 ],
       [2663.419922]])

In [35]:
m1_prediction = scaler.inverse_transform(Xm1)
m2_prediction = scaler.inverse_transform(Xm2)
m3_prediction = scaler.inverse_transform(Xm3)

print(m1_prediction)
print(m2_prediction)
print(m3_prediction)

[[2669.7715]
 [2675.5442]
 [2676.4587]
 [2673.1802]
 [2661.179 ]
 [2652.547 ]
 [2649.0513]
 [2654.5254]
 [2664.8352]]
[[2674.5562]
 [2643.1008]
 [2624.3406]
 [2624.8604]
 [2638.9268]
 [2650.847 ]
 [2660.235 ]
 [2660.8306]
 [2654.3035]]
[[2694.5269]
 [2691.9736]
 [2681.83  ]
 [2670.035 ]
 [2659.6846]
 [2656.6204]
 [2659.4873]
 [2667.1084]
 [2675.0427]]


In [36]:
m2_Y_validation

array([[2634.560059],
       [2639.399902],
       [2666.939941],
       [2669.909912],
       [2648.050049],
       [2654.800049],
       [2635.669922],
       [2629.72998 ],
       [2663.419922]])

# level 0 model 2 validation unscaled output => m2_prediction

In [ ]:
# getting test data output

In [37]:
Xmt1=model1.predict(X_test)
Xmt1.shape

(9, 1)

In [38]:
Xmt2=model2.predict(X_test)
Xmt2.shape

(9, 1)

In [39]:
Xmt3=model3.predict(X_test)
Xmt3.shape

(9, 1)

In [40]:
Y_test.reshape(-1,1)
Y_test.shape

(9,)

In [41]:
m1_x_test = scaler.inverse_transform(Xmt1)
m2_x_test = scaler.inverse_transform(Xmt2)
m3_x_test = scaler.inverse_transform(Xmt3)

In [42]:
m2_x_test

array([[2697.4011],
       [2701.308 ],
       [2706.2556],
       [2705.451 ],
       [2700.9854],
       [2697.285 ],
       [2692.017 ],
       [2693.0186],
       [2691.9978]], dtype=float32)

In [43]:
m1_x_test

array([[2730.5303],
       [2726.1597],
       [2712.208 ],
       [2697.4966],
       [2697.891 ],
       [2704.8904],
       [2704.684 ],
       [2706.3137],
       [2698.5588]], dtype=float32)

In [44]:
m3_x_test

array([[2735.932 ],
       [2731.5208],
       [2724.3442],
       [2718.6926],
       [2720.848 ],
       [2725.4446],
       [2723.8516],
       [2723.3909],
       [2716.6282]], dtype=float32)

In [45]:
Y_test = Y_test.reshape(-1,1)
m2_Y_test = scaler.inverse_transform(Y_test)

In [46]:
m2_Y_test

array([[2733.01001 ],
       [2724.439941],
       [2733.290039],
       [2727.76001 ],
       [2721.330078],
       [2689.860107],
       [2724.01001 ],
       [2705.27002 ],
       [2734.620117]])

In [47]:
from sklearn.metrics import mean_squared_error

level0_model2_test_mse = mean_squared_error(m2_x_test, m2_Y_test)
print("level 0 model 2 test MSE ", level0_model2_test_mse)

level 0 model 2 test MSE  721.2191672747989


In [48]:
level0_model1_test_mse = mean_squared_error(m1_x_test, m2_Y_test)
print("level 0 model 1 test MSE ", level0_model1_test_mse)

level 0 model 1 test MSE  424.41383545185846


In [49]:
level0_model3_test_mse = mean_squared_error(m3_x_test, m2_Y_test)
print("level 0 model 3 test MSE ", level0_model3_test_mse)

level 0 model 3 test MSE  237.72305994450335


# now train level 1 model with level 0 models' output

In [50]:
Y_validation

array([[0.18350648],
       [0.20008866],
       [0.29444584],
       [0.3046215 ],
       [0.22972564],
       [0.25285236],
       [0.18730908],
       [0.16695776],
       [0.28238561]])

In [51]:
m2_Y_validation

array([[2634.560059],
       [2639.399902],
       [2666.939941],
       [2669.909912],
       [2648.050049],
       [2654.800049],
       [2635.669922],
       [2629.72998 ],
       [2663.419922]])

In [59]:
import tensorflow as tf
from tensorflow import keras
model4 = keras.Sequential([
    keras.layers.Flatten(input_shape=(3,)),
    keras.layers.Dense(18, activation='relu'),
    keras.layers.Dense(9, activation='relu'),
    #keras.layers.Dense(4, activation='relu'),
    # yucen: change from 2 to 3
    keras.layers.Dense(3, activation='relu'),
    keras.layers.Dense(1)
])

In [60]:
model4.compile(optimizer='rmsprop', 
              loss='mean_squared_error',
              metrics=['mae'])

In [61]:
scaler = MinMaxScaler()
normalized_x_validation1 = scaler.fit_transform(m1_prediction)
normalized_x_validation2 = scaler.fit_transform(m2_prediction)
normalized_x_validation3 = scaler.fit_transform(m3_prediction)

normalized_y_validation = scaler.fit_transform(m2_Y_validation)

In [62]:
level1_train_data=np.concatenate((normalized_x_validation1, normalized_x_validation2, normalized_x_validation3), axis=-1)
level1_train_data.shape

(9, 3)

In [63]:
model4.fit(level1_train_data, normalized_y_validation, epochs=100, batch_size=2)

Train on 9 samples
Epoch 1/100
9/9 [==============================] - 0s 17ms/sample - loss: 0.2818 - mean_absolute_error: 0.4121
Epoch 2/100
9/9 [==============================] - 0s 3ms/sample - loss: 0.2573 - mean_absolute_error: 0.3894
Epoch 3/100
9/9 [==============================] - 0s 2ms/sample - loss: 0.2414 - mean_absolute_error: 0.3799
Epoch 4/100
9/9 [==============================] - 0s 2ms/sample - loss: 0.2281 - mean_absolute_error: 0.3646
Epoch 5/100
9/9 [==============================] - 0s 1ms/sample - loss: 0.2127 - mean_absolute_error: 0.3622
Epoch 6/100
9/9 [==============================] - 0s 2ms/sample - loss: 0.2034 - mean_absolute_error: 0.3584
Epoch 7/100
9/9 [==============================] - 0s 1ms/sample - loss: 0.1926 - mean_absolute_error: 0.3505
Epoch 8/100
9/9 [==============================] - 0s 2ms/sample - loss: 0.1814 - mean_absolute_error: 0.3492
Epoch 9/100
9/9 [==============================] - 0s 2ms/sample - loss: 0.1729 - mean_absolute_erro

In [64]:
normalized_x_test1 = scaler.fit_transform(m1_x_test)
normalized_x_test2 = scaler.fit_transform(m2_x_test)
normalized_x_test3 = scaler.fit_transform(m3_x_test)

In [65]:
level1_test_data=np.concatenate((normalized_x_test1, normalized_x_test2, normalized_x_test3), axis=-1)
level1_test_data.shape

(9, 3)

In [66]:
Xt4 =model4.predict(level1_test_data)
Xt4.shape

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


(9, 1)

In [67]:
predicted_stock_price = scaler.inverse_transform(Xt4)

In [68]:
mse = mean_squared_error(predicted_stock_price, m2_Y_test)
print("Blending Ensemble MSE: ", mse)

Blending Ensemble MSE:  183.80227310243868


In [69]:
predicted_stock_price

array([[2724.4119],
       [2726.4148],
       [2721.1138],
       [2719.3037],
       [2718.6453],
       [2720.253 ],
       [2723.203 ],
       [2723.593 ],
       [2725.17  ]], dtype=float32)